## Основы глубокого обучения в NLP. CharRNN

План на сегодня: RNN - генератор имён

1. Базовая работа с текстом: токенизация, кодирование и декодирование
2. Рекуррентные архитектуры: RNN, LSTM, GRU
3. Обучение генерации == next token prediction

### 1. Готовим данные

In [ ]:
# ! wget https://download.pytorch.org/tutorial/data.zip
# ! unzip data.zip

In [ ]:
! head -n 5 data/names/Chinese.txt

#### 1.1. Пишем датасет и компоновщик батчей

Нам нужно:
1. Прочитать все имена из текстовых файлов
2. Закодировать каждое имя как последовательность целых чисел, предварительно добавив к именам символы начала и окончания (зачем?)
3. Сохранить пары (список токенов, id языка)
4. Седать разбиение на train/test
5. Реализовать сборку примеров в батчи

In [ ]:
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch
from torch import Tensor, nn
import torch.nn.functional as F

In [ ]:
class NamesDataset(Dataset):
    # псевдоним для пары имя-язык
    _ItemPair = tuple[str, int]

    vocabulary: dict[str, int]
    languages: dict[str, int]
    names: list[_ItemPair]

    def __init__(self, datadir: Path) -> None:
        pad_token = ''
        bos_token = '?'  # beginning of sequence
        eos_token = '\n'  # end of sequence
        self.vocabulary = {pad_token: 0, bos_token: 1, eos_token: 2}
        self.languages = {}
        self.names = []
        # iterate over files, update vocabulary, save name + language pairs
        ...

    @property
    def vocab_size(self) -> int:
        return len(self.vocabulary)
    
    @property
    def num_classes(self) -> int:
        return len(self.languages)

    def encode(self, name: str) -> list[int]:
        ...
    
    def decode(self, encoded: list[int]) -> str:
        ...

    def __getitem__(self, index: int) -> tuple[list[int], int]:
        return self.names[index]
    
    def __len__(self) -> int:
        return len(self.names)

Проверка:

In [ ]:
dataset = NamesDataset(Path("data/names/"))
tokens, label = dataset[4444]
print(tokens, label)
print(dataset.decode(tokens), dataset.languages[label])


Разбивка датасета на трейн и тест:

In [ ]:
from copy import deepcopy

def train_test_split(dataset: NamesDataset, ratio: float = 0.1) -> tuple[NamesDataset, NamesDataset]:
    ...


In [ ]:
train_dataset, test_dataset = train_test_split(dataset, ratio=0.1)
print("Train size: ", len(train_dataset))
print("Test size: ", len(test_dataset))

Упаковка в батчи:

In [ ]:
def collate_fn(batch: list[tuple[list[int], int]]) -> tuple[Tensor, Tensor]:
    ...

In [ ]:
batch = [train_dataset[i] for i in range(8)]
tokens, labels = collate_fn(batch)
print("Tokens shape: ", tokens.shape, "\nLabels shape: ", labels.shape)

Собираем в загрузчик данных:

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
tokens, labels = next(iter(train_loader))
print("Tokens shape: ", tokens.shape, "\nLabels shape: ", labels.shape)

### 2. Пишем простую RNN

Начнём с написания RNNCell - одного рекуррентного блока

<img src="https://i.stack.imgur.com/02KvP.png" style="background:white" width="600"/>

<!-- <img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" style="background:white" width="600"/> -->



In [ ]:
class RNNCell(nn.Module):
    """
    (x_{t}, h_{t-1}) -> h_{t}
    """
    def __init__(self, input_dim: int, hidden_dim: int) -> None:
        super().__init__()
        ...


Проверка:

In [ ]:
batch_size = 4
input_dim = 10
hidden_dim = 8
cell = RNNCell(input_dim, hidden_dim)
h = torch.randn(1, hidden_dim)
# расширяем до размеров батча
h_expanded = h.expand((batch_size, -1))
x = torch.randn(batch_size, input_dim)
h_new = cell.forward(x, h_expanded)
print(h_new.shape)

**Упражнение 1**: реализуйте более сложно устроенную LSTMCell, где теперь есть:
1. два внутренних состояния: cell state $c_t$ и hidden state $h_t$
2. набор гейтов для управления обновлениями состояний

[blog post](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
<!-- <img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" style="background:white" width="600"/> -->

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png" style="background:white" width="500"/>
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png" style="background:white" width="500"/>
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png" style="background:white" width="500"/>
<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png" style="background:white" width="500"/>

**Упражнение 2**: реализуйте GRUCell

<img src="https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png" style="background:white" width="500"/>

Опишем модель, состоящую из следующих блоков:
1. `embed`: кодирует входные токены в векторы размера `hidden_dim`
2. `rnn`: наша рекуррентная ячейка
3. `output`: восстанавливает логиты из скрытого состояния `h`

In [ ]:

class RNN(nn.Module):
    def __init__(self, vocab_size: int, hidden_dim: int) -> None:
        super().__init__()
        ...


    def forward(self, x: Tensor) -> Tensor:
        ...


**Упражнение 3**. Добавьте в класс `RNN` возможность
   1. Нескольких последовательных рекуррентных слоёв
   2. Выбора другого типа рекуррентной ячейки (`GRU`, `LSTM`)

Проверка:

In [ ]:
hidden_dim = 32
model = RNN(
    vocab_size=train_dataset.vocab_size,
    hidden_dim=hidden_dim,
)
model.forward(tokens).shape

### 3. Функция для генерации

Схема:
1. Подаём на вход произвольный префикс имени, можно только токен начала
2. Проходимся моделью по префиксу, получаем логиты для следующего токена
3. Семплируем новый токен, добавляем его к префиксу, возвращаемся к шагу 1.
4. Критерии остановки:
   - встретили символ окончания строки
   - сгенерировали максимальное число новых токенов

In [ ]:
@torch.no_grad()
def generate(model: nn.Module, idx: Tensor, max_new_tokens: int) -> Tensor:
    ...


**Упражнение 4**. Модифицируйте функцию `generate`, чтобы она при семплировании учитывала
   - $k$ наиболее вероятных токенов (параметр `top_k: int`)
   - только токены, дающие в сумме вероятность не меньше $p$ (параметр `top_p: int`)
   - температуру для `softmax`:


      $\begin{aligned}\text{softmax}(x_i, \tau) = \frac{\exp(x_i / \tau)}{\sum_j \exp(x_i / \tau)} \end{aligned}$

Ещё понадобится функция, которая умеет декодировать выход из функции `generate`

In [ ]:
def batch_decode(out_tokens: Tensor) -> list[str]:
    ...
        

Сгенерируем несколько "имён" для проверки, начиная со $\texttt{<BOS>}$ токена:

In [ ]:
samples = generate(model, idx=torch.full(size=(4, 1), fill_value=1, dtype=int), max_new_tokens=40)
print('\n'.join(batch_decode(samples)))

### 4. Цикл обучения

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(20):
    ...


In [ ]:
samples = generate(model, idx=torch.full(size=(4, 1), fill_value=1, dtype=int), max_new_tokens=40)
print('\n'.join(batch_decode(samples)))

### Упражнения

5. Модифицируйте вычисление ошибки, чтобы не считать её для токенов, отвечающих за паддинг. Повлияло ли это на скорость обучения модели?
6. Добавьте в генерацию входное условие: язык для генерируемого имени
7. Используйте `nn.LSTM` и `nn.GRU` вместо самописных моделей, сравните результаты. 
8. Реализуйте модель для классификации имён по языкам